# Build a Dashboard Application with Plotly Dash

In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in real-time.

This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

*   TASK 1: Add a Launch Site Drop-down Input Component
*    TASK 2: Add a callback function to render __success-pie-chart__ based on selected site dropdown
*    TASK 3: Add a Range Slider to Select Payload
*    TASK 4: Add a callback function to render the __success-payload-scatter-chart__ scatter plot

After visual analysis using the dashboard, you should be able to obtain some insights to answer the following five questions:

*    Which site has the largest successful launches?
*    Which site has the highest launch success rate?
*    Which payload range(s) has the highest launch success rate?
*    Which payload range(s) has the lowest launch success rate?
*    Which F9 Booster version (__v1.0, v1.1, FT, B4, B5,__ etc.) has the highest
    launch success rate?

## Downloading and Prepping Data

If you are using local jupyter lab, then  add these lines in your code:

In [1]:
"""
!pip install dash
!pip install dash==1.19.0
!pip install jupyter_dash
!pip install --upgrade plotly
!pip install jupyter-dash
"""

'\n!pip install dash\n!pip install dash==1.19.0\n!pip install jupyter_dash\n!pip install --upgrade plotly\n!pip install jupyter-dash\n'

## Read the Data

Let's start with

* Importing necessary libraries
* Reading the data

In [2]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
import dash
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
import plotly.express as px
from dash.dependencies import Input, Output

# Read the SpaceX data into pandas dataframe
spacex_df=pd.read_csv("spacex_launch_dash.csv")

spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
max_payload=spacex_df["Payload Mass (kg)"].max()
min_payload=spacex_df["Payload Mass (kg)"].min()

print("Maximum payload mass ",max_payload)
print("Mainimum payload mass",min_payload)

Maximum payload mass  9600.0
Mainimum payload mass 0.0


## TASK 1: Add a Launch Site Drop-down Input Component

We have four different launch sites and we would like to first see which one has the largest success count. Then, we would like to select one specific site and check its detailed success rate (class=0 vs. class=1).

As such, we will need a dropdown menu to let us select different launch sites.

Find and complete a commented __dcc.Dropdown(id='site-dropdown',...)__ input with following attributes:

*    __id__ attribute with value __site-dropdown__
*    __options__ attribute is a list of dict-like option objects (with label and value attributes). You can set the label and value all to be the launch  site names in the spacex_df and you need to include the default All option. e.g.,

  `options=[{'label': 'All Sites', 'value': 'ALL'},{'label': 'site1', 'value': 'site1'}, ...]`
  
*    __value__ attribute with default dropdown value to be ALL meaning all sites are selected
*    __placeholder__ attribute to show a text description about this input area, such as Select a Launch Site here
*    __searchable__ attribute to be True so we can enter keywords to search launch sites

In [4]:
# Create a dash application

#app = dash.Dash(__name__)
#This line used for JupyterLab
app= JupyterDash(__name__)


# Build dash app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                style={'textAlign': 'center', 'color': '#503D36', 'font-size': 30}),
                                
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                            ],
                                             value= 'ALL',
                                             placeholder='Select a Launch Site here',
                                             searchable=True
                                             # style={'width':'80%','padding':'3px','font-size':'20px','text-align-last':'center'}
                                            ),
                                html.Br(),
                                
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                
                                
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                html.P("Payload range (Kg):"),
                                dcc.RangeSlider(id='payload-slider',
                                               min=0, max=10000, step=1000,
                                               #marks={0: '0', 100 : '100'},
                                               value=[min_payload, max_payload]),
                                
                                
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))

                                ])

## TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown

The general idea of this callback function is to get the selected launch site from site-dropdown and render a pie chart visualizing launch success counts.

Dash callback function is a type of Python function which will be automatically called by Dash whenever receiving an input component updates, such as a click or dropdown selecting event.

Let's add a callback function in __spacex_dash_app.py__ including the following application logic:

*    Input is set to be the __site-dropdown__ dropdown, i.e., __Input(component_id='site-dropdown', component_property='value')__
*    Output to be the graph with id __success-pie-chart__, i.e., __Output(component_id='success-pie-chart', component_property='figure')__
*    A __If-Else__ statement to check if ALL sites were selected or just a specific launch site was selected

*    If ALL sites are selected, we will use all rows in the dataframe spacex_df to render and return a pie chart graph to show the total success launches (i.e., the total count of class column)
*    If a specific launch site is selected, you need to filter the dataframe spacex_df first in order to include the only data for the selected site. Then, render and return a pie chart graph to show the success (__class=1__) count and failed (__class=0__) count for the selected site.

## TASK 3: Add a Range Slider to Select Payload

Next, we want to find if variable payload is correlated to mission outcome. From a dashboard point of view, we want to be able to easily select different payload range and see if we can identify some visual patterns.

Find and complete a commented __dcc.RangeSlider(id='payload-slider',...)__ input with the following attribute:

*    __id__ to be __payload-slider__
*    __min__ indicating the slider starting point, we set its value to be 0 (Kg)
*    __max__ indicating the slider ending point to, we set its value to be 10000 (Kg)
*    __step__ indicating the slider interval on the slider, we set its value to be 1000 (Kg)
*    __value__ indicating the current selected range, we could set it to be __min_payload__ and __max_payload__

In [5]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', names='Launch Site',
                    title='Total Success Launches By Site')
        return fig
    
    else:
        # return the outcomes piechart for a selected site
        filtered_df=spacex_df[spacex_df['Launch Site']== entered_site]
        filtered_df=filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig=px.pie(filtered_df,values='class count',names='class',title=f"Total Success Launches for site {entered_site}")
        return fig

## TASK 4: Add a callback function to render the __success-payload-scatter-chart__ scatter plot

Next, we want to plot a scatter plot with the x axis to be the payload and the y axis to be the launch outcome (i.e., __class__ column). As such, we can visually observe how payload may be correlated with mission outcomes for selected site(s).

In addition, we want to color-label the Booster version on each scatter point so that we may observe mission outcomes with different boosters.

Now, let's add a call function including the following application logic:

*    Input to be __[Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")]__ Note that we have two input components, one to receive selected launch site and another to receive selected payload range
*    Output to be __Output(component_id='success-payload-scatter-chart', component_property='figure')__
*    A __If-Else__ statement to check if ALL sites were selected or just a specific launch site was selected
*    If ALL sites are selected, render a scatter plot to display all values for variable __Payload Mass (kg)__ and variable __class__.

In addition, the point color needs to be set to the booster version i.e., __color="Booster Version Category"__

*    If a specific launch site is selected, you need to filter the __spacex_df__ first, and render a scatter chart to show values __Payload Mass (kg)__ and __class__ for the selected site, and color-label the point using __Boosster Version Category__ likewise.

In [6]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
                [Input(component_id='site-dropdown',component_property='value'),
                Input(component_id='payload-slider',component_property='value')])

def scatter(entered_site,payload):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])]
    # thought reusing filtered_df may cause issues, but tried it out of curiosity and it seems to be working fine
    
    if entered_site=='ALL':
        fig=px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category',
                       title='Success count on Payload mass for all sites')
        return fig
    
    else:
        fig=px.scatter(filtered_df[filtered_df['Launch Site']==entered_site],x='Payload Mass (kg)', y='class',color='Booster Version Category',
                       title=f"Success count on Payload mass for site {entered_site}")
        return fig

    
# Run the app
if __name__ == '__main__':
    # Run app and display result inline in the notebook
    #app.run_server(mode='inline')
    
    #For JupyterLab
    app.run_server(mode='jupyterlab', port = 8090, dev_tools_ui=True, debug=True,dev_tools_hot_reload =True, threaded=True)

Later in the browser address bar use this

http://localhost:8090

### Finding Insights Visually

Now with the dashboard completed, you should be able to use it to analyze SpaceX launch data, and answer the following questions:

*    Which site has the largest successful launches?
*    Which site has the highest launch success rate?
*    Which payload range(s) has the highest launch success rate?
*    Which payload range(s) has the lowest launch success rate?
*    Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?

                                                        © IBM Corporation 2021. All rights reserved.